In [32]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt  # Pour générer des couleurs dynamiquesfrom matplotlib.colors import rgb2hex 
from matplotlib.colors import rgb2hex



/tmp/ipykernel_41490/2330095304.py:11: DtypeWarning:

Columns (19,62,63,76,77,79,80,81,82,83,84,85,86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.



In [28]:
# Charger les données
# Charger les données
path = "ECHANTILLON_BASE_SIV_ENTREPRISES_FINALE.csv"
data = pd.read_csv(path)


Index(['Id', 'siren', 'Nom_entreprise_titulaire', 'siret',
       'activite_principale', 'SIREN_du_titulaire', 'SIREN_du_locataire',
       'b2_lou_siren', 'location', 'date_creation', 'date_debut_activite',
       'date_fermeture', 'tranche_effectif_salarie',
       'annee_tranche_effectif_salarie', 'geo_adresse', 'geo_id', 'latitude',
       'longitude', 'adresse', 'code_postal', 'commune', 'libelle_commune',
       'b1_loc_prenom', 'b1_tit_adr_type_voie', 'b1_tit_adr_nom_voie',
       'b1_tit_adr_code_postal', 'b1_tit_adr_commune', 'b1_loc_adr_type_voie',
       'b1_loc_adr_nom_voie', 'b1_loc_adr_code_postal', 'b1_loc_adr_commune',
       'b3_nb_cotitulaires', 'b5_tvv', 'b5_type_recep', 'b5_vin', 'b5_cnit',
       'b5_masse_f1', 'b5_masse_f2', 'b5_masse_f3', 'b5_masse_f4',
       'b5_poids_a_vide', 'b5_categorie', 'b5_genre', 'b5_carrosserie',
       'b5_num_reception', 'b5_cylindree', 'b5_puissance_net_maxi',
       'b5_energie', 'b5_puissance_admin', 'b5_nb_places_assises',
      

In [33]:
data['Annee_observation'] = pd.to_datetime(data['Annee_observation'])

In [34]:
# Filtrer le DataFrame avec les critères de sélection
df = pd.DataFrame()
df['Annee_observation'] = data['Annee_observation'].dt.year
df['Statut_propriété'] =data['Statut_propriété']
df['Statut_propriété']= data['Statut_propriété']
df['secteur_activite_principale']= data['secteur_activite_principale']
df['crit_air'] = data["crit_air"]
df['group_category'] = data['group_category']




# Créer le DataFrame final filtré
df_structured = pd.DataFrame(df)

In [35]:
df_structured

,Annee_observation,Statut_propriété,secteur_activite_principale,crit_air,group_category
0,2016,locataire,Transport,NaN,Inconnu
1,2016,locataire,Location et location-bail,Crit'Air 2,Poids lourds
2,2016,locataire,Construction,Crit'Air 2,Véhicules utilitaires légers
3,2016,locataire,Autre,Crit'Air 2,Véhicules utilitaires légers
4,2016,locataire,Construction,Crit'Air 2,Véhicules utilitaires légers
...,...,...,...,...,...
515174,2022,propriétaire,Location et location-bail,Crit'Air 2,Véhicules utilitaires légers
515175,2022,propriétaire,Location et location-bail,Crit'Air 2,Véhicules utilitaires légers
515176,2022,propriétaire,Location et location-bail,Crit'Air 2,Véhicules utilitaires légers
515177,2022,propriétaire,Autre,Crit'Air 3,Voitures particulières


In [36]:
# Créer le DataFrame final filtré
df_structured = pd.DataFrame(df)

# Création de l'application Dash
app = dash.Dash(__name__)

# Layout de l'application Dash
app.layout = html.Div([
    html.H1("Analyse de crit_air par secteur d'activité"),
    
    # Dropdown pour sélectionner l'année
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': str(year), 'value': year} for year in df_structured['Annee_observation'].unique()],
        value=df_structured['Annee_observation'].min(),
        clearable=False
    ),
    
    # Dropdown pour sélectionner le status de propriété
    dcc.Dropdown(
        id='status-dropdown',
        options=[],
        clearable=False
    ),
    
    # Dropdown pour sélectionner le secteur d'activité
    dcc.Dropdown(
        id='sector-dropdown',
        options=[],
        clearable=False
    ),
    
    # Dropdown pour sélectionner la catégorie de véhicule
    dcc.Dropdown(
        id='catégorie-dropdown',
        options=[],
        clearable=False
    ),
    
    # Graphique camembert
    dcc.Graph(id='pie-chart')
])

# Callback pour mettre à jour les options de status en fonction de l'année
@app.callback(
    Output('status-dropdown', 'options'),
    Input('year-dropdown', 'value'))
def set_status_options(selected_year):
    status_options = df_structured[df_structured['Annee_observation'] == selected_year]['Statut_propriété'].unique()
    return [{'label': status, 'value': status} for status in status_options]

# Callback pour mettre à jour les options de secteur en fonction du status et de l'année
@app.callback(
    Output('sector-dropdown', 'options'),
    [Input('year-dropdown', 'value'),
     Input('status-dropdown', 'value')])
def set_sector_options(selected_year, selected_status):
    if selected_status is None:
        return []
    
    filtered_data = df_structured[
        (df_structured['Annee_observation'] == selected_year) & 
        (df_structured['Statut_propriété'] == selected_status)
    ]
    sector_options = filtered_data['secteur_activite_principale'].unique()
    return [{'label': sector, 'value': sector} for sector in sector_options]

# Callback pour mettre à jour les options de catégorie de véhicule en fonction des filtres précédents
@app.callback(
    Output('catégorie-dropdown', 'options'),
    [Input('year-dropdown', 'value'),
     Input('status-dropdown', 'value'),
     Input('sector-dropdown', 'value')])
def set_category_options(selected_year, selected_status, selected_sector):
    if selected_sector is None:
        return []
    
    filtered_data = df_structured[
        (df_structured['Annee_observation'] == selected_year) & 
        (df_structured['Statut_propriété'] == selected_status) & 
        (df_structured['secteur_activite_principale'] == selected_sector)
    ]
    category_options = filtered_data['group_category'].unique()
    return [{'label': category, 'value': category} for category in category_options]

# Callback pour afficher le graphique camembert en fonction des filtres
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('year-dropdown', 'value'),
     Input('status-dropdown', 'value'),
     Input('sector-dropdown', 'value'),
     Input('catégorie-dropdown', 'value')])
def update_pie_chart(selected_year, selected_status, selected_sector, selected_category):
    if selected_sector is None or selected_category is None:
        return {}
    
    filtered_data = df_structured[
        (df_structured['Annee_observation'] == selected_year) & 
        (df_structured['Statut_propriété'] == selected_status) & 
        (df_structured['secteur_activite_principale'] == selected_sector) &
        (df_structured['group_category'] == selected_category)
    ]
    
    if filtered_data.empty:
        return {}
    
    # Récupérer les valeurs uniques de crit_air
    unique_crit_air = filtered_data["crit_air"].unique()
    
    # Générer dynamiquement les couleurs en fonction du nombre de valeurs uniques de crit_air
    num_unique = len(unique_crit_air)
    cmap = plt.cm.get_cmap('tab10', num_unique)  # Utiliser 'tab10' pour générer une palette de couleurs
    colors = [rgb2hex(cmap(i)) for i in range(num_unique)]  # Utiliser matplotlib.colors.rgb2hex pour convertir en hexadécimal

    # Associer chaque crit_air à une couleur différente
    color_map = {crit_air: colors[i] for i, crit_air in enumerate(unique_crit_air)}
    
    # Créer le camembert avec plotly express et appliquer le mapping de couleurs
    fig = px.pie(filtered_data, 
                 names='crit_air', 
                 title=f'Répartition crit_air - Année {selected_year}, Status {selected_status}, Secteur {selected_sector}, Catégorie {selected_category}',
                 hole=0.3,  # Paramètre 'hole' pour un graphique en donut (optionnel)
                 color='crit_air',  # Définir la colonne de couleur
                 color_discrete_map=color_map)  # Appliquer le mapping de couleurs

    # Personnalisation des légendes
    fig.update_layout(
        legend_title_text='Catégorie Crit Air',
        legend=dict(
            orientation="v",  # 'v' pour vertical, 'h' pour horizontal
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.05  # Ajustement pour placer la légende à droite du graphique
        )
    )
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)